In [4]:
!pip install xdem
!pip install pyproj
!pip install rasterio
!pip install IPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.6/708.6 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.2 MB/s eta 0:00:00


In [7]:
import requests
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from PIL import Image
from IPython.display import display
import pprint
import xdem
import matplotlib.pyplot as plt
import pyproj
from io import BytesIO
import math
import rasterio

In [8]:
def print_tif_details(tif_path):
    with rasterio.open(tif_path) as src:
        print(f"\n📁 File: {tif_path}")
        print("=" * 50)

        # Basic Info
        print(f"🧾 Driver: {src.driver}")
        print(f"📐 Width x Height: {src.width} x {src.height}")
        print(f"🗂️  Band count: {src.count}")
        print(f"🧊 Data type: {src.dtypes[0]}")
        print(f"🚫 NoData value: {src.nodata}")

        # Coordinate System
        print("\n🌍 Coordinate Reference System (CRS):")
        print(src.crs)

        # Affine transform (pixel -> coordinate)
        print("\n📏 Affine Transform:")
        print(src.transform)

        # Bounds
        print("\n🧭 Bounding Box:")
        print(f"Left: {src.bounds.left}")
        print(f"Bottom: {src.bounds.bottom}")
        print(f"Right: {src.bounds.right}")
        print(f"Top: {src.bounds.top}")

        # Resolution
        print("\n🔎 Pixel Size:")
        print(f"X: {src.res[0]}, Y: {src.res[1]}")

        # Metadata
        print("\n📝 Metadata Tags:")
        pprint.pprint(src.tags())

        # Band-specific info
        for i in range(1, src.count + 1):
            stats = src.statistics(i, approx=True)
            print(f"\n📊 Band {i} Statistics (approx):")
            print(f"  Min: {stats.min}, Max: {stats.max}")
            print(f"  Mean: {stats.mean}, StdDev: {stats.std}")

In [9]:
def get_epsg_from_latlon(latitude, longitude):
    """
    Get the EPSG code for a given latitude and longitude.

    Parameters:
    latitude (float): Latitude value.
    longitude (float): Longitude value.

    Returns:
    str: EPSG code.
    """
    # Calculate UTM zone number
    zone_number = int((longitude + 180) / 6) + 1

    epsg_code = 32600 + zone_number if latitude >= 0 else 32700 + zone_number

    return f"{epsg_code}"

In [10]:
def margin_from_zoom(lat, zoom, image_size_pixels):
    """
    Calculate margin in degrees to match Google Maps zoom level for given latitude and image size.

    This uses the Web Mercator projection formula that Google Maps uses.
    Returns a margin in degrees for both latitude and longitude.
    """

    # Google Maps uses Web Mercator projection (EPSG:3857)
    # At zoom level z, the map shows a region of width 360/2^z degrees longitude
    # and a corresponding height in latitude (which varies with latitude due to Mercator projection)

    # For a 256x256 tile at zoom level z, the tile covers:
    # - Longitude: 360 / (2^z) degrees
    # - The latitude coverage depends on the Mercator projection

    # Calculate the ground resolution (meters per pixel) at the given latitude and zoom
    # This is the standard Web Mercator formula
    cos_lat = math.cos(math.radians(lat))
    ground_resolution = (156543.03392 * cos_lat) / (2 ** zoom)

    # Convert the image size from pixels to meters
    image_width_meters = image_size_pixels * ground_resolution
    image_height_meters = image_size_pixels * ground_resolution

    # Convert meters to degrees
    # Latitude: 1 degree ≈ 111,320 meters (constant)
    meters_per_degree_lat = 111320.0
    margin_lat = image_height_meters / (2 * meters_per_degree_lat)

    # Longitude: varies with latitude
    meters_per_degree_lon = meters_per_degree_lat * cos_lat
    margin_lon = image_width_meters / (2 * meters_per_degree_lon)

    return margin_lat, margin_lon

In [11]:
def generate_dem(lat_center, lon_center, API_KEY, zoom=13, image_size_pixels = 600, dem_path = "dem.tif", demtype = "COP30" , ):

    margin_lat, margin_lon = margin_from_zoom(lat_center, zoom, image_size_pixels)

    print(f"   margin: {margin_lat, margin_lon}")


    # --- CONFIGURATION ---
    params = {
        "demtype": demtype,  # Options: SRTMGL1, COP30, etc.
        "south": lat_center - margin_lat,
        "north": lat_center + margin_lat,
        "west": lon_center - margin_lon,
        "east": lon_center + margin_lon,
        "outputFormat": "GTiff",
        "API_Key": API_KEY
    }

    # --- DOWNLOAD DEM ---
    url = "https://portal.opentopography.org/API/globaldem"
    response = requests.get(url, params=params)

    with open(dem_path, "wb") as f:
        f.write(response.content)

    # --- PROCESS DEM ---
    dem = xdem.DEM(dem_path)

    # Set the vertical CRS
    if dem.vcrs is None:
        dem.set_vcrs("EGM08")

    dem.set_nodata(-9999)

    #collect the target_crs_epsg
    target_crs_epsg = get_epsg_from_latlon(lat_center, lon_center)
    print(f"Target CRS EPSG: {target_crs_epsg}")

     # Define the target projected CRS
    target_crs = pyproj.CRS.from_epsg(target_crs_epsg)

    # Reproject the DEM to the target CRS
    dem = dem.reproject(crs=target_crs, inplace=False)

    # Save the processed DEM
    dem.save(dem_path)

    print("DEM file saved at:", dem_path)

    return dem_path

In [12]:
def print_tif_image(dem_path):
    # --- READ DEM ---
    with rasterio.open(dem_path) as src:
        elevation = src.read(1)  # First band
        nodata = src.nodata

    # --- MASK NODATA ---
    elevation = np.ma.masked_equal(elevation, nodata)

    # --- NORMALIZE + APPLY COLORMAP ---
    norm = Normalize(vmin=np.min(elevation), vmax=np.max(elevation))
    normalized = norm(elevation)
    cmap = plt.cm.viridis
    colored = cmap(normalized)[:, :, :3]  # Drop alpha

    # --- CONVERT TO 8-BIT RGB IMAGE ---
    img_rgb = (colored * 255).astype(np.uint8)
    img_pil = Image.fromarray(img_rgb)

    # --- DISPLAY IMAGE ---
    plt.figure(figsize=(10, 10))
    plt.title("DEM Heatmap")
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

    # --- SAVE AS JPEG ---
    img_pil.save("dem_heatmap.jpg", "JPEG")
    print("Saved as dem_heatmap.jpg")

In [13]:
def visualize_terrain_attributes(dem_path, what = "all"):
    """
    Visualize various terrain attributes from a DEM using xDEM.

    Parameters:
    dem_path (str): Path to the DEM file.

    Returns:
    None
    """
    # Load the DEM
    dem = xdem.DEM(dem_path)

    # Compute terrain attributes
    slope =  xdem.terrain.slope(dem, method="ZevenbergThorne")
    aspect = dem.aspect()
    hillshade = dem.hillshade()
    curvature = dem.curvature()

    # Set up the plot
    fig, axs = plt.subplots(2, 2, figsize=(7, 7)) if( what == "all") else plt.subplots(figsize=(10, 10))

    if what == "slope":
        slope.plot(ax=axs, cmap='Reds', cbar_title='Slope (°)')
        axs.set_title('Slope')

    if  what=="aspect":
        # Plot Aspect
        aspect.plot(ax=axs, cmap='twilight', cbar_title='Aspect (°)')
        axs.set_title('Aspect')

    if  what=="hillshade":
        # Plot Hillshade
        hillshade.plot(ax=axs, cmap='gray', cbar_title='Hillshade')
        axs.set_title('Hillshade')

    if what=="curvature":
        # Plot Curvature
        curvature.plot(ax=axs, cmap='coolwarm', cbar_title='Curvature')
        axs[1, 1].set_title('Curvature')

    # Adjust layout and display
    plt.tight_layout()
    plt.show()

In [14]:
def visualize_slope(dem_path):

    """
    Visualize various terrain attributes from a DEM using xDEM.

    Parameters:
    dem_path (str): Path to the DEM file.

    Returns:
    None
    """

    # Load the DEM
    dem = xdem.DEM(dem_path)

    # Compute terrain attributes
    slope =  xdem.terrain.slope(dem, method="ZevenbergThorne")

    # Set up the plot
    fig, axs = plt.subplots(figsize=(10, 10))

    slope.plot(ax=axs, cmap='Reds', cbar_title='Slope (°)')
    axs.set_title('Slope')

    # Adjust layout and display
    plt.tight_layout()
    plt.show()

In [18]:

# lat_center, lon_center =  45.7045520559693, 9.858910394898183
# zoom_level = 10
# image_size_pixels = 256

# OPENTOPO_APIK = "hahahhahah... what?"

# print("Downloading DEM...")
# dem_file = generate_dem(lat_center, lon_center, OPENTOPO_APIK, zoom_level, image_size_pixels)


In [16]:
# visualize_slope(dem_file)

# print("printing details...")
# print_tif_details(dem_file)

# # print("plotting...")
# print_tif_image(dem_file)

# # print("Visualizing terrain attributes...")
# visualize_terrain_attributes(dem_file, what="slope")

